# Downloading the data

ask linus: linus says the data is public and can be downloaded. 

(EU data should be public but costs 800 euros :( from https://publication-bdds.apps.epo.org/raw-data/products)

google search: `uspto data largest full text download mirror archive`

find https://developer.uspto.gov/data

right click and copy link

In [1]:
!wget https://bulkdata.uspto.gov/data/patent/application/redbook/2024/I20240606.tar

--2024-06-11 12:33:50--  https://bulkdata.uspto.gov/data/patent/application/redbook/2024/I20240606.tar
Resolving bulkdata.uspto.gov (bulkdata.uspto.gov)... 151.207.245.28
Connecting to bulkdata.uspto.gov (bulkdata.uspto.gov)|151.207.245.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5267940864 (4.9G) [application/x-tar]
Saving to: ‘I20240606.tar’

I20240606.tar       100%[===================>]   4.91G  3.70MB/s    in 25m 40s 

2024-06-11 12:59:30 (3.26 MB/s) - ‘I20240606.tar’ saved [5267940864/5267940864]



In [4]:
!ls -lh I20240606.tar

-rw-r--r--  1 me  staff   4.9G Jun  6 06:02 I20240606.tar


In [3]:
!mkdir -p ~/data/uspto.gov

In [4]:
!tar -xvf I20240606.tar -C ~/data/uspto.gov

x I20240606/PLANT/US20240188459P1-20240606.ZIP
x I20240606/PLANT/US20240188469P1-20240606.ZIP
x I20240606/PLANT/US20240188460P1-20240606.ZIP
x I20240606/PLANT/US20240188461P1-20240606.ZIP
x I20240606/PLANT/US20240188463P1-20240606.ZIP
x I20240606/PLANT/US20240188468P1-20240606.ZIP
x I20240606/PLANT/US20240188465P1-20240606.ZIP
x I20240606/PLANT/US20240188462P1-20240606.ZIP
x I20240606/PLANT/US20240188458P1-20240606.ZIP
x I20240606/PLANT/US20240188467P1-20240606.ZIP
x I20240606/PLANT/US20240188466P1-20240606.ZIP
x I20240606/PLANT/US20240188464P1-20240606.ZIP
x I20240606/UTIL0184/US20240184924A1-20240606.ZIP
x I20240606/UTIL0184/US20240184624A1-20240606.ZIP
x I20240606/UTIL0184/US20240184450A1-20240606.ZIP
x I20240606/UTIL0184/US20240184597A1-20240606.ZIP
x I20240606/UTIL0184/US20240184262A1-20240606.ZIP
x I20240606/UTIL0184/US20240184012A1-20240606.ZIP
x I20240606/UTIL0184/US20240184414A1-20240606.ZIP
x I20240606/UTIL0184/US20240184739A1-20240606.ZIP
x I20240606/UTIL0184/US20240184446A1

In [3]:
!ls -lh ~/data/uspto.gov/

total 0
drwxr-xr-x  14 me  staff   448B Jun 12 02:08 I20240606/
drwxr-xr-x   6 me  staff   192B Jun 11 13:05 I20240606-SUPP/


# Load data using code written by a large language model

In [6]:
%load_ext jupyter_ai

In [7]:
%load_ext dotenv

In [8]:
%dotenv

In [9]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

Take this folder: 

```
total 0
drwxr-xr-x  14 me  staff   448B Jun 12 02:08 I20240606/
drwxr-xr-x   6 me  staff   192B Jun 11 13:05 I20240606-SUPP/
```

This contains a nested hierarchy of ZIP files: 

This is partial output from this command (remember the path! it's important): 

```
!ls -lh ~/data/uspto.gov/
```

Output:

```
x I20240606/UTIL0183/US20240183228A1-20240606.ZIP
x I20240606/UTIL0183/US20240183126A1-20240606.ZIP
x I20240606/UTIL0183/US20240183146A1-20240606.ZIP
x I20240606/UTIL0183/US20240183199A1-20240606.ZIP
x I20240606-SUPP/UTIL0180/US20240180980A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0180/US20240180983A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0180/US20240180975A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0180/US20240180964A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0180/US20240180974A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182876A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182556A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182905A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182928A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182894A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182874A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0181/US20240181029A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0181/US20240181084A1-20240606-SUPP.ZIP
x I20240606-SUPP/DTDS/DTDS.ZIP
```

Every ZIP file has one or more XML files in the standard USPTO xml format.

Please write a short python snippet using as few lines of code as possible using standard canonical reference files from github or other reputable sources with many stars, using standard libraries using runtime static type checking (beartype library), type-checking, in order to export the data from XML format to parquet format.

debug it. proceed step-by-step as an elite site reliability/devops/L20 principal warez engineer at google, returning as few tokens as possible, to debug this dbt/analytics engineering code. give the complete corrected code! your code will be directly fed to a hungry jupyter notebook cell and should have no comupction or superfluous behavior. at all :)

In [11]:
import os
from zipfile import ZipFile
from typing import List
from beartype import beartype
from lxml import etree
import pyarrow as pa
import pyarrow.parquet as pq

@beartype
def process_zip_files(base_dir: str, output_dir: str) -> None:
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".ZIP"):
                zip_path = os.path.join(root, file)
                process_zip_file(zip_path, output_dir)

@beartype
def process_zip_file(zip_path: str, output_dir: str) -> None:
    with ZipFile(zip_path, 'r') as zip_file:
        xml_files = [f for f in zip_file.namelist() if f.endswith('.xml')]
        for xml_file in xml_files:
            with zip_file.open(xml_file) as xml_data:
                xml_content = xml_data.read()
                patent_data = parse_xml(xml_content)
                write_to_parquet(patent_data, output_dir)

@beartype
def parse_xml(xml_content: bytes) -> List[dict]:
    root = etree.fromstring(xml_content)
    patent_data = []
    for patent in root.xpath('//us-patent-grant'):
        patent_id = patent.find('.//publication-reference/document-id/doc-number').text
        title = patent.find('.//invention-title').text
        # Parse other required fields
        patent_data.append({
            'patent_id': patent_id,
            'title': title,
            # Add other fields
        })
    return patent_data

@beartype
def write_to_parquet(patent_data: List[dict], output_dir: str) -> None:
    table = pa.Table.from_pylist(patent_data)
    pq.write_table(table, os.path.join(output_dir, 'patent_data.parquet'))

if __name__ == '__main__':
    base_dir = '~/data/uspto.gov/'
    output_dir = 'output/'
    os.makedirs(output_dir, exist_ok=True)
    process_zip_files(base_dir, output_dir)

In [12]:
!ls -lh ./output

total 0


## Second attempt: give more context

In [21]:
code = r"""
import os
from zipfile import ZipFile
from typing import List
from beartype import beartype
from lxml import etree
import pyarrow as pa
import pyarrow.parquet as pq

@beartype
def process_zip_files(base_dir: str, output_dir: str) -> None:
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".ZIP"):
                zip_path = os.path.join(root, file)
                process_zip_file(zip_path, os.path.join(output_dir, os.path.splitext(file)[0]))

@beartype
def process_zip_file(zip_path: str, output_dir: str) -> None:
    with ZipFile(zip_path, 'r') as zip_file:
        xml_files = [f for f in zip_file.namelist() if f.endswith('.xml')]
        for xml_file in xml_files:
            with zip_file.open(xml_file) as xml_data:
                xml_content = xml_data.read()
                patent_data = parse_xml(xml_content)
                write_to_parquet(patent_data, output_dir)

@beartype
def parse_xml(xml_content: bytes) -> List[dict]:
    root = etree.fromstring(xml_content)
    patent_data = []
    for patent in root.xpath('//us-patent-grant'):
        patent_id = patent.find('.//publication-reference/document-id/doc-number').text
        title = patent.find('.//invention-title').text
        # Parse other required fields
        patent_data.append({
            'patent_id': patent_id,
            'title': title,
            # Add other fields
        })
    return patent_data

@beartype
def write_to_parquet(patent_data: List[dict], output_dir: str) -> None:
    os.makedirs(output_dir, exist_ok=True)
    table = pa.Table.from_pylist(patent_data)
    pq.write_table(table, os.path.join(output_dir, 'patent_data.parquet'))

base_dir = '~/data/uspto.gov'
output_dir = 'output'
process_zip_files(base_dir, output_dir)
"""

In [32]:
parent_prompt = f"""

Take this folder: 

```
total 0
drwxr-xr-x  14 me  staff   448B Jun 12 02:08 I20240606/
drwxr-xr-x   6 me  staff   192B Jun 11 13:05 I20240606-SUPP/
```

This contains a nested hierarchy of ZIP files: 

```
x I20240606/UTIL0183/US20240183228A1-20240606.ZIP
x I20240606/UTIL0183/US20240183126A1-20240606.ZIP
x I20240606/UTIL0183/US20240183146A1-20240606.ZIP
x I20240606/UTIL0183/US20240183199A1-20240606.ZIP
x I20240606-SUPP/UTIL0180/US20240180980A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0180/US20240180983A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0180/US20240180975A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0180/US20240180964A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0180/US20240180974A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182876A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182556A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182905A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182928A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182894A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0182/US20240182874A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0181/US20240181029A1-20240606-SUPP.ZIP
x I20240606-SUPP/UTIL0181/US20240181084A1-20240606-SUPP.ZIP
x I20240606-SUPP/DTDS/DTDS.ZIP
```

Every ZIP file has one or more XML files.

Use this code as a starter pack / expansion pack for your maximum capabilities! 

```
{{code}}
```

Print progress at every step for every file processed, and make sure you start from this directory: 

```
~/data/uspto.gov
```

And traverse it completely for every ZIP file recursively within, and collects the FULL TEXT of every single file patent description and invention description etc (will be used in vector search db).

Make sure to first print out all of the ZIP files you are about to traverse recursively using a simple print statement :)

Also make sure to use the absolute paths everywhere!
"""

In [33]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt} 

Make sure there is output and you have checked that there is one parquet file per folder (or subfolder, however you decide to shard it using best practices from analytics engineering :)

Please write a short python snippet using standard libraries using runtime static type checking (beartype library), type-checking, in order to export the data from XML format to parquet format.

debug it. proceed step-by-step as an elite site reliability/devops/L20 principal warez engineer at google, returning as few tokens as possible, to debug this dbt/analytics engineering code. give the complete corrected code! your code will be directly fed to a hungry jupyter notebook cell and should have no comupction or superfluous behavior. at all :)

In [34]:
import os
from zipfile import ZipFile
from typing import List
from beartype import beartype
from lxml import etree
import pyarrow as pa
import pyarrow.parquet as pq

@beartype
def process_zip_files(base_dir: str, output_dir: str) -> None:
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".ZIP"):
                zip_path = os.path.join(root, file)
                print(f"Processing ZIP file: {zip_path}")
                process_zip_file(zip_path, os.path.join(output_dir, os.path.splitext(file)[0]))

@beartype
def process_zip_file(zip_path: str, output_dir: str) -> None:
    with ZipFile(zip_path, 'r') as zip_file:
        xml_files = [f for f in zip_file.namelist() if f.endswith('.xml')]
        for xml_file in xml_files:
            print(f"Processing XML file: {xml_file}")
            with zip_file.open(xml_file) as xml_data:
                xml_content = xml_data.read()
                patent_data = parse_xml(xml_content)
                write_to_parquet(patent_data, output_dir)

@beartype
def parse_xml(xml_content: bytes) -> List[dict]:
    root = etree.fromstring(xml_content)
    patent_data = []
    for patent in root.xpath('//us-patent-grant'):
        patent_id = patent.find('.//publication-reference/document-id/doc-number').text
        title = patent.find('.//invention-title').text
        description = ' '.join(patent.xpath('.//description/p/text()'))
        claims = ' '.join(patent.xpath('.//claims/claim/claim-text/text()'))
        patent_data.append({
            'patent_id': patent_id,
            'title': title,
            'description': description,
            'claims': claims
        })
    return patent_data

@beartype
def write_to_parquet(patent_data: List[dict], output_dir: str) -> None:
    os.makedirs(output_dir, exist_ok=True)
    table = pa.Table.from_pylist(patent_data)
    pq.write_table(table, os.path.join(output_dir, 'patent_data.parquet'))

base_dir = os.path.expanduser('~/data/uspto.gov')
output_dir = os.path.expanduser('~/output')
process_zip_files(base_dir, output_dir)

Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181977A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181962A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181302A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181317A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181716A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181703A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181861A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181874A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181399A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL0181/US20240181798A1-20240606.ZIP
Processing ZIP file: /Users/me/data/uspto.gov/I20240606/UTIL

In [37]:
!ls -lh output

total 0
